# Listen to FM Synthesis

*Marina Bosi & Rich Goldberg*<br>
*Center for Computer Research in Music and Acoustics (CCRMA)* 

In FM Synthesis, two sinusoids are combined such that one of them modulates the instantaneous frequency of the other one.  If the modulating sinusoid is $\sin(2 \pi f_m  t)$ (where its frequency is  $f_m$) then the output signal x(t) is
$$x(t) = \sin\left( 2 \pi f_c t + m \ \sin(2 \pi f_m  t)\right)$$
where $f_c$ is the frequency of the sinusoid being modulated and $m$ is the modulation index describing the degree of modulation. 

The mathematics of frequency modulation tell us the the resulting frequency spectrum will be a line at the frequency $f_c$ and other lines spaced integer multiples of $f_m$ away from $f_c$ where the amplitude of the line $n$ multiples away from $f_c$ will equal $J_n(m)$ where the function $J_n(x)$ is a "Bessel Function of the First Kind".  Hence, the combination of these two sinusoids will create a spectrum with mutliple lines it.  

Further, in cases where $m$ is large enough so that audible lines pass into negative frequencies, they are perceived as having positive frequency and so frequency lines that are no longer multiples of $f_m$ away from $f_c$ appear in the spectrum, allowing for very complex sounds.  (For example, if $f_c$ were $1 kHz$ and $f_m$ were $300 Hz$ then, in addition to the possibility of lines at \{$100Hz, 400Hz, 700Hz, 1kHz, 1.3kHz, ...\}, the negative frequencies could mix in lines at \{200Hz, 500Hz, 800Hz, 1.1 kHz, ...\}.  Hence, FM Synthesis allows for the creation of very complex sounds from only two oscillators.

The widgets below allow you to set the key FM Synthesis parameters $f_c, f_m, m$ and then both listen to the result and see its frequency spectrum.  

In [1]:
#HIDDEN
# Setup
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display
from ipywidgets import interactive, fixed  
# ipwidgets makes it easy to make Jupyter notebooks interactive
#  * interactive() for making function inputs interactive
#  * fixed() allows params to be fixed


In [2]:
#HIDDEN
# Plot the Fourier transform of (mono) signal sampled at sample_rate
def PlotFFT(signal, sample_rate, Nfft = 2048, start = 1024, log_scale = True):
    """ Plot the Fourier transform of (mono) `signal` that is sampled 
    at `sample_rate`.  Uses an `Nfft` point DFT for data starting at  
    sample number `start`.  (Does not return anything, just makes the plot.)
    """
    w = np.hanning(Nfft)
    x_fft = np.fft.fft(w * signal[start: start + Nfft])[:Nfft//2]
    f_fft = np.arange(Nfft//2) * sample_rate/Nfft
    if log_scale:
        plt.xlim(100, 20_000)
        plt.semilogx(f_fft, 10.*np.log10( np.abs( x_fft )**2 *4 * 8/3 /Nfft**2))
    else:
        plt.xlim(0, 20_000)
        plt.plot(f_fft, 10.*np.log10( np.abs( x_fft )**2 *4 * 8/3 /Nfft**2))
    plt.ylim(-80,10)
    plt.xlabel("Frequency in Hz")
    plt.ylabel("dB")
    plt.title("DFT of Signal")
    plt.gca().get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
    plt.show()

In [ ]:
#HIDDEN
# Carry out FM synthesis and show the result 
def fm_synth(fc = 2_000, fm = 1_500, m = 5., max_time = 3., rate = 44_100, log_scale = True):
    """Creates an FM modulated signal where the carrier has frequency `fc`,
    the modulating signal has frequency `fm`, and the modulation index is `m`.
    It then plays `max_time` seconds of the signal sampled with sample rate
    `rate` and displays if DFT (computed using `PlotFFT()` defined above.)
    """
    times = np.linspace(0,max_time,rate*max_time+1)
    signal = np.sin(2*np.pi*fc*times + m * np.sin(2*np.pi*fm*times))
    display(Audio(data=signal, rate=rate, autoplay = True))
    PlotFFT(signal, rate, log_scale=log_scale)
    return signal

In [3]:
#HIDDEN
# interactively run the function `fm_synth()`
w = interactive( fm_synth
               , fc=(500, 5_000)
               , fm=(500, 5_000)
               , m = (0., 30.)
               , max_time = fixed(3)
               , rate = fixed(44_100)
               , log_scale = True
               )
display(w)

interactive(children=(IntSlider(value=2000, description='fc', max=5000, min=500), IntSlider(value=1500, descri…